In [1]:
import pandas as pd
from dataclasses import dataclass, asdict, replace, astuple
import requests
import json

In [2]:
def submit_data(collection, docs, credential):
    headers = {'Content-Type': 'application/json', 'Access-Control-Request-Headers': '*','api-key': credential['key']}
    insert_url = f"{credential['uri']}/action/insertMany"
    Payload = json.dumps({"collection": collection, "database": credential['db'], "dataSource": credential['cluster'], "documents": docs})
    response = requests.request("POST", insert_url, headers=headers, data=Payload)
    return response
    
def get_data(collection, credential):
    headers = {'Content-Type': 'application/json', 'Access-Control-Request-Headers': '*','api-key': credential['key']}
    findAll_url =  f"{credential['uri']}/action/find"
    Payload = json.dumps({"collection": collection, "database":credential['db'], "dataSource": credential['cluster'], "filter": {}, "limit":5000})
    response = requests.request("POST", findAll_url, headers=headers, data=Payload)
    response_json = response.json()['documents']
    # status = [_['status'] for _ in response_json]
    df = pd.read_json(json.dumps(response_json))
    return df


def update_data(collection, condition, doc,  credential):
    headers = {'Content-Type': 'application/json', 'Access-Control-Request-Headers': '*','api-key': credential['key']}
    updateOne_url = f"{credential['uri']}/action/updateOne"
    Payload = json.dumps({"collection": collection, "database": credential['db'], "dataSource": credential['cluster'], "filter": condition, "update":{"$set": doc}})
    response = requests.request("POST", updateOne_url, headers=headers, data=Payload)
    return response

In [8]:
wwl_db = {'cluster': "WWL", 
'uri': 'https://data.mongodb-api.com/app/data-qvnrx/endpoint/data/v1' ,
'db': "wwl" ,
'key': "5d9sGO28viSiX1HnJlOLN6QMqPqxYz6NIKVUMvEU8wXvAS0CPHMMHs2jF0UHKSCF"}

In [3]:
@dataclass
class Attendance:
    present: str  
    name: str 
    department: str

In [7]:
name_df = pd.read_excel('LWW Ice Attendance.xlsx',sheet_name='Sheet2')
dep_lt = name_df['duty'].values.tolist()
name_lt = name_df['name'].values.tolist()
list_name = []
for i in range (name_df.shape[0]):
    record = Attendance(present= 'No', name=name_lt[i] , department=dep_lt[i] )
    list_name.append(asdict(record))
list_name

[{'present': 'No', 'name': 'Guruwinder Singh', 'department': 'Duty Manager'},
 {'present': 'No',
  'name': 'El Mostafa Ben Mouina',
  'department': 'Duty Manager'},
 {'present': 'No', 'name': 'Morris Mbugu', 'department': 'Duty Manager'},
 {'present': 'No',
  'name': 'Thembani Tywakadi',
  'department': 'Ice Rink Supervisor'},
 {'present': 'No', 'name': 'Abdul Samad', 'department': 'Ice Rink Supervisor'},
 {'present': 'No', 'name': 'Aleksandra Antczak', 'department': 'Ice Marshall'},
 {'present': 'No', 'name': 'Izabella Magdowska', 'department': 'Ice Marshall'},
 {'present': 'No', 'name': 'Konrad Kita', 'department': 'Ice Marshall'},
 {'present': 'No',
  'name': 'Ethelmae Jewel Suguitan',
  'department': 'Ice Marshall'},
 {'present': 'No', 'name': 'Gabija Meilunaite', 'department': 'Ice Marshall'},
 {'present': 'No',
  'name': 'Abdulrahman Mohhamed Ahmed Albahri',
  'department': 'Ice Marshall'},
 {'present': 'No', 'name': 'Jose Fernando', 'department': 'Ice Marshall'},
 {'present': 'N

In [9]:
submit_data('attendance', list_name, wwl_db)

<Response [201]>